In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [2]:
# Data for the table
data = {
    'Plant': ['Plant A', 'Plant B', 'Plant C', 'Plant D', 'Plant E'],
    'DC1': [180, 110, 145, 180, 135],
    'DC2': [215, 175, 165, 220, 165],
    'DC3': [65, 95, 150, 175, 160],
    'DC4': [55, 145, 95, 365, 65],
    'DC5': [105, 165, 150, 190, 130],
    'DC6': [110, 120, 180, 185, 150],
    'DC7': [100, 80, 195, 155, 10000],
    'DC8': [125, 160, 120, 265, 10000],
    'DC9': [155, 205, 135, 290, 10000]
}

# Create a DataFrame
costs = pd.DataFrame(data)
costs.set_index('Plant', inplace=True)


In [3]:
# Capacities at plants
capacities = {
    'Plant A': 1500,
    'Plant B': 800,
    'Plant C': 1190,
    'Plant D': 2900,
    'Plant E': 1250
}

# Requirements at DCs
requirements = {
    'DC1': 520,
    'DC2': 800,
    'DC3': 300,
    'DC4': 1450,
    'DC5': 375,
    'DC6': 100,
    'DC7': 935,
    'DC8': 750,
    'DC9': 620
}


no_plant_e = ['DC7', 'DC8', 'DC9']

In [4]:
m = gp.Model()
x = m.addVars(capacities.keys(), requirements.keys(), lb=0, name="Proportion")
m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2026-01-17


In [5]:
demand = m.addConstrs((gp.quicksum(x[i, j] for i in capacities.keys()) >= requirements[j] for j in requirements.keys()), name="Demand")
capacity = m.addConstrs((gp.quicksum(x[i, j] for j in requirements.keys()) <= capacities[i] for i in capacities.keys()), name="Capacity")
no_shipment = m.addConstrs((x['Plant E', dc] == 0 for dc in no_plant_e), name="No Shipment Plant E to DC 7, 8, 9")
m.update()

In [6]:
m.setObjective(gp.quicksum(costs.loc[i, j] * x[i, j] for i in capacities.keys() for j in requirements.keys()), GRB.MINIMIZE)
m.update()

In [7]:
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 17 rows, 45 columns and 93 nonzeros
Model fingerprint: 0x18c60654
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+01, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 3e+03]
Presolve removed 3 rows and 3 columns
Presolve time: 0.02s
Presolved: 14 rows, 42 columns, 84 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.312500e+02   0.000000e+00      0s
      18    6.6882500e+05   0.000000e+00   0.000000e+00      0s

Solved in 18 iterations and 0.04 seconds (0.00 work units)
Optimal objective  6.688250000e+05


In [8]:
for v in m.getVars():
    if v.X > 0:
        print(v.VarName, v.X)


print("Objective Value (Total Cost): ")
print(m.ObjVal)

Proportion[Plant A,DC3] 300.0
Proportion[Plant A,DC4] 545.0
Proportion[Plant A,DC5] 375.0
Proportion[Plant A,DC6] 100.0
Proportion[Plant A,DC8] 180.0
Proportion[Plant B,DC7] 800.0
Proportion[Plant C,DC8] 570.0
Proportion[Plant C,DC9] 620.0
Proportion[Plant D,DC1] 520.0
Proportion[Plant D,DC2] 455.0
Proportion[Plant D,DC7] 135.0
Proportion[Plant E,DC2] 345.0
Proportion[Plant E,DC4] 905.0
Objective Value (Total Cost): 
668825.0


In [11]:
results = pd.DataFrame(index=capacities.keys(), columns=requirements.keys())

# Populate the DataFrame with variable values
for i in capacities.keys():
    for j in requirements.keys():
        results.loc[i, j] = x[i, j].X

results

,DC1,DC2,DC3,DC4,DC5,DC6,DC7,DC8,DC9
Plant A,0.0,0.0,300.0,545.0,375.0,100.0,0.0,180.0,0.0
Plant B,0.0,0.0,0.0,0.0,0.0,0.0,800.0,0.0,0.0
Plant C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,570.0,620.0
Plant D,520.0,455.0,0.0,0.0,0.0,0.0,135.0,0.0,0.0
Plant E,0.0,345.0,0.0,905.0,0.0,0.0,0.0,0.0,0.0


In [14]:
pi_data = {constr.ConstrName: constr.Pi for constr in m.getConstrs()}
pi = pd.Series(pi_data)
pi

Demand[DC1]                               180.0
Demand[DC2]                               220.0
Demand[DC3]                               130.0
Demand[DC4]                               120.0
Demand[DC5]                               170.0
Demand[DC6]                               175.0
Demand[DC7]                               155.0
Demand[DC8]                               190.0
Demand[DC9]                               205.0
Capacity[Plant A]                         -65.0
Capacity[Plant B]                         -75.0
Capacity[Plant C]                         -70.0
Capacity[Plant D]                           0.0
Capacity[Plant E]                         -55.0
No Shipment Plant E to DC 7, 8, 9[DC7]      0.0
No Shipment Plant E to DC 7, 8, 9[DC8]      0.0
No Shipment Plant E to DC 7, 8, 9[DC9]      0.0
dtype: float64

In [19]:
for v in m.getVars():
    print(v.VarName, v.RC)

Proportion[Plant A,DC1] 65.0
Proportion[Plant A,DC2] 60.0
Proportion[Plant A,DC3] 0.0
Proportion[Plant A,DC4] 0.0
Proportion[Plant A,DC5] 0.0
Proportion[Plant A,DC6] 0.0
Proportion[Plant A,DC7] 10.0
Proportion[Plant A,DC8] 0.0
Proportion[Plant A,DC9] 15.0
Proportion[Plant B,DC1] 5.0
Proportion[Plant B,DC2] 30.0
Proportion[Plant B,DC3] 40.0
Proportion[Plant B,DC4] 100.0
Proportion[Plant B,DC5] 70.0
Proportion[Plant B,DC6] 20.0
Proportion[Plant B,DC7] 0.0
Proportion[Plant B,DC8] 45.0
Proportion[Plant B,DC9] 75.0
Proportion[Plant C,DC1] 35.0
Proportion[Plant C,DC2] 15.0
Proportion[Plant C,DC3] 90.0
Proportion[Plant C,DC4] 45.0
Proportion[Plant C,DC5] 50.0
Proportion[Plant C,DC6] 75.0
Proportion[Plant C,DC7] 110.0
Proportion[Plant C,DC8] 0.0
Proportion[Plant C,DC9] 0.0
Proportion[Plant D,DC1] 0.0
Proportion[Plant D,DC2] 0.0
Proportion[Plant D,DC3] 45.0
Proportion[Plant D,DC4] 245.0
Proportion[Plant D,DC5] 20.0
Proportion[Plant D,DC6] 10.0
Proportion[Plant D,DC7] 0.0
Proportion[Plant D,DC8]